In [1]:
import os
import re
from glob import glob
import pandas as pd

from IPython.display import display, Markdown

## 0. Configuratie

In [4]:
course_id = "netwerkanalyse"
assignment_id = "tentamen1"

## 1. nbgrader configureren
code-stub overschrijven etc.

In [3]:
config = '''
c = get_config()

c.Exchange.course_id = "%s"
c.ClearSolutions.code_stub = {'python': '# JOUW CODE HIER', 'markdown': '....'}
''' % course_id

with open('nbgrader_config.py', 'w') as f:
    f.write(config)

## 2. Assignment-map aanmaken

In [11]:
!nbgrader db assignment add {assignment_id}
!mkdir -p source/{assignment_id}

!ls -l

# Met curl template ophalen

display(Markdown('## 3. Assignment maken\nHet notebook-template staat in **source/%s/**  \n<a href="/notebooks/source/%s/%s.ipynb" target="_blank">Open notebook</a>' % (assignment_id, assignment_id, assignment_id)))


[DbAssignmentAddApp | INFO] Creating/updating assignment with ID 'tentamen1': {'duedate': None}
total 320
drwxr-xr-x@ 3 Jesse  staff      96 Apr  3 13:10 autograded
-rw-r--r--@ 1 Jesse  staff  135168 Apr  3 13:29 gradebook.db
-rw-r--r--@ 1 Jesse  staff   11567 Apr  3 13:38 nbgrader workflow.ipynb
-rw-r--r--@ 1 Jesse  staff     119 Apr  3 12:45 nbgrader_config.py
drwxr-xr-x@ 8 Jesse  staff     256 Apr  3 13:08 original-submissions
drwxr-xr-x@ 4 Jesse  staff     128 Apr  3 13:06 release
drwxr-xr-x@ 4 Jesse  staff     128 Apr  3 12:45 source
-rw-r--r--@ 1 Jesse  staff     212 Apr  3 13:29 student-results.csv
-rw-r--r--@ 1 Jesse  staff    4634 Apr  3 13:08 submissions.zip
drwxr-xr-x@ 4 Jesse  staff     128 Apr  3 13:09 submitted


## 3. Assignment maken
Het notebook-template staat in **source/tentamen1/**  
<a href="/notebooks/source/tentamen1/tentamen1.ipynb" target="_blank">Open notebook</a>

## 4. Maak een studentversie

In [5]:
!rm -rf release
!nbgrader assign {assignment_id} --create

student_nb = glob('release/%s/*.ipynb' % assignment_id)[0]
student_nb_name = os.path.basename(student_nb)

print("\nDe studentversie bevindt zich in %s" % student_nb)

[AssignApp | INFO] Updating/creating assignment 'tentamen1': {}
[AssignApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/source/./tentamen1/Tentamen1.ipynb
[AssignApp | INFO] Writing 3562 bytes to /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/release/./tentamen1/Tentamen1.ipynb
[AssignApp | INFO] Setting destination file permissions to 644

De studentversie bevindt zich in release/tentamen1/Tentamen1.ipynb


## Link naar studentversie in Markdown

## Tentamen afnemen

## 4. Zet mappenstructuur inzendingen op

In [6]:
submissions_zip = 'submissions.zip'

!unzip -j {submissions_zip} -d "original-submissions"

Archive:  submissions.zip
  inflating: original-submissions/Tentamen1_Jesse_vd_Sar_10735291.ipynb  
  inflating: original-submissions/._Tentamen1_Jesse_vd_Sar_10735291.ipynb  
  inflating: original-submissions/Tentamen1-checkpoint.ipynb  
  inflating: original-submissions/._Tentamen1-checkpoint.ipynb  
  inflating: original-submissions/._.ipynb_checkpoints  
  inflating: original-submissions/._submissions  


In [7]:
nbs_copied = 0

for f in glob('original-submissions/*.ipynb'):
    nb = os.path.basename(f)
    
    ids = re.findall(r'\d{5,}', nb)
    if len(ids):
        student_id = ids[0]
        
        !mkdir -p submitted/{student_id}/{assignment_id}
        !cp {f} submitted/{student_id}/{assignment_id}/{student_nb_name}
        
        nbs_copied += 1
        
print("%d notebook(s) gevonden en gekopiëerd" % nbs_copied)

1 notebook(s) gevonden en gekopiëerd


## 5. Autograden

In [8]:
!nbgrader autograde {assignment_id} --create

[AutogradeApp | INFO] Creating/updating student with ID '10735291': {}
[AutogradeApp | INFO] Overwriting files with master versions from the source directory
[AutogradeApp | INFO] Sanitizing /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/submitted/10735291/tentamen1/Tentamen1.ipynb
[AutogradeApp | INFO] Converting notebook /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/submitted/10735291/tentamen1/Tentamen1.ipynb
[AutogradeApp | WARNING] Attribute 'checksum' for cell vraag-1-test has changed! (should be: 1d1b73ce516a9d7a8e0bad4c4f053e95, got: 3028f70e2c94f75426dbe4c4521aa5e0)
[AutogradeApp | INFO] Writing 3910 bytes to /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/autograded/10735291/tentamen1/Tentamen1.ipynb
[AutogradeApp | INFO] Autograding /Users/Jesse/Dropbox/Notebook-exams/notebook-exams/Scripts/nbgrader/Voorbeeld/autograded/10735291/tentamen1/Tentamen1.ipynb
[AutogradeApp | INFO] Convert

## Link naar manual graden

## 6. students-answers.csv genereren

In [36]:
import numpy as np

q_oud = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit,
        comment.auto_comment,
        comment.manual_comment
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
        INNER JOIN solution_cell ON solution_cell.name = grade_cell.name AND solution_cell.notebook_id = grade_cell.notebook_id
        LEFT JOIN comment ON comment.notebook_id = grade.notebook_id AND comment.cell_id = solution_cell.id
'''

q = '''
    SELECT
        submitted_assignment.student_id,
        grade_cell.name AS question_name,
        grade_cell.max_score,
        grade.needs_manual_grade AS needs_grading,
        grade.auto_score,
        grade.manual_score,
        grade.extra_credit
    FROM grade
        INNER JOIN submitted_notebook ON submitted_notebook.id = grade.notebook_id
        INNER JOIN submitted_assignment ON submitted_assignment.id = submitted_notebook.assignment_id
        INNER JOIN grade_cell ON grade_cell.id = grade.cell_id
'''

df = pd.read_sql_query(q, 'sqlite:///gradebook.db')
df['final_score'] = np.where(~np.isnan(df['manual_score']), df['manual_score'], df['auto_score']) + df['extra_credit'].fillna(0)

df.to_csv('student-results.csv')

## 7. Analyseren

In [ ]:
per_student = df.groupby('student_id')

per_student['final_score'].sum() / per_student['max_score'].sum()